<center>

<h1 style="text-align:center"> Generalized Algebraic Data Types </h1>
<h2 style="text-align:center"> CS3100 Fall 2019 </h2>
</center>

## Simple language

Consider this simple language of integers and booleans

In [77]:
type value =
  | Int of int
  | Bool of bool

type expr =
  | Val of value
  | Plus of expr * expr
  | Mult of expr * expr
  | Ite of expr * expr * expr

type value = Int of int | Bool of bool


type expr =
    Val of value
  | Plus of expr * expr
  | Mult of expr * expr
  | Ite of expr * expr * expr


## Evaluator for the simple language

We can write a simple evaluator for this language

In [78]:
let rec eval : expr -> value =
  fun e -> match e with
  | Val (Int i) -> Int i
  | Val (Bool i) -> Bool i
  | Plus (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 + i2)
  | Mult (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 * i2)
  | Ite (p,e1,e2) ->
    let Bool b = eval p in
    if b then eval e1 else eval e2  

File "[78]", line 6, characters 4-62:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Int _, Bool _)|(Bool _, _))
File "[78]", line 9, characters 4-62:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Int _, Bool _)|(Bool _, _))
File "[78]", line 12, characters 4-61:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
Int _


val eval : expr -> value = <fun>


## Evaluator for the simple language

* The compiler complains that programs such as `true + 10` is not handled.
  + Our evaluator gets **stuck** when it encouters such an expression.

In [63]:
eval @@ Plus (Val (Bool true), Val (Int 10))

error: compile_error

* We need **Types**
  + Well-typed programs do not get stuck!

## Phantom types

* We can add types to our values using a technique called **phantom types**

In [79]:
type 'a value =
  | Int of int
  | Bool of bool

type 'a value = Int of int | Bool of bool


* Observe that `'a` only appears on the LHS.
  + This `'a` is called a phantom type variable.
* What is this useful for?

## Typed expression language

We can add types to our expression language now using phantom type

In [80]:
type 'a expr =
  | Val of 'a value
  | Plus of int expr * int expr
  | Mult of int expr * int expr
  | Ite of bool expr * 'a expr * 'a expr

(* Assign concerte type to the phantom type variable 'a*)
let mk_int i : int expr = Val (Int i)
let mk_bool b : bool expr = Val (Bool b)
let plus e1 e2 : int expr = Plus (e1, e2)
let mult e1 e2 : int expr = Mult (e1, e2)

type 'a expr =
    Val of 'a value
  | Plus of int expr * int expr
  | Mult of int expr * int expr
  | Ite of bool expr * 'a expr * 'a expr


val mk_int : int -> int expr = <fun>


val mk_bool : bool -> bool expr = <fun>


val plus : int expr -> int expr -> int expr = <fun>


val mult : int expr -> int expr -> int expr = <fun>


## Benefit of phantom types

In [81]:
let i = Val (Int 0);;
let i' = mk_int 0;;

let b = Val (Bool true);;
let b' = mk_bool true;;

let p = Plus (i,i);;
let p' = plus i i;;

val i : 'a expr = Val (Int 0)


val i' : int expr = Val (Int 0)


val b : 'a expr = Val (Bool true)


val b' : bool expr = Val (Bool true)


val p : 'a expr = Plus (Val (Int 0), Val (Int 0))


val p' : int expr = Plus (Val (Int 0), Val (Int 0))


## Benefit of phantom types

We no longer allow ill-typed expression if we use the helper functions.

In [70]:
plus (mk_bool true) (mk_int 10) 

error: compile_error

## Typed evaluator

We can write an evaluator for this language now.

Let's use the same evaluator as the earlier one.

In [83]:
let rec eval : 'a expr -> 'a value = 
  fun e -> match e with
  | Val (Int i) -> Int i
  | Val (Bool i) -> Bool i
  | Plus (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 + i2)
  | Mult (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 * i2)
  | Ite (p,e1,e2) ->
    let Bool b = eval p in
    if b then eval e1 else eval e2

File "[83]", line 6, characters 4-62:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Int _, Bool _)|(Bool _, _))


error: compile_error

File "[83]", line 9, characters 4-62:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Int _, Bool _)|(Bool _, _))


## Typed evaluator

* OCaml by default expects the function expression at the recursive call position to have the same type as the outer function.
* This need not be the case if the function is polymorphic
  + `eval (p : int expr)` and `eval (p : bool expr)`.
* In order to allow this, OCaml supports polymorphic recursion (aka Milner-Mycroft typeability)
  + Robin Milner co-invented type infererence + polymorphism that we using in OCaml.
  + Alan Mycroft was my mentor at Cambridge :-) 

## Fixing the interpreter with polymorphic recursion

In [87]:
let rec eval : type a. a expr -> a value = 
  fun e -> match e with
  | Val (Int i) -> Int i
  | Val (Bool i) -> Bool i
  | Plus (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 + i2)
  | Mult (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 * i2)
  | Ite (p,e1,e2) ->
    let Bool b = eval p in
    if b then eval e1 else eval e2

File "[87]", line 6, characters 4-62:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Int _, Bool _)|(Bool _, _))
File "[87]", line 9, characters 4-62:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Int _, Bool _)|(Bool _, _))
File "[87]", line 12, characters 4-61:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
Int _


val eval : 'a expr -> 'a value = <fun>


## Errors gone, but warning remains

* Compiler still warns us that there are unhandled cases in pattern matches
* But haven't we added types to the expression language?
* Observe that `mk_int i = Val (Int i)` is just convention.
  + You can still write ill-typed expression by directly using the constructors.

In [89]:
eval @@ Plus (Val (Bool true), Val (Int 10))

error: runtime_error

* Here, `Val (Bool true)` is inferred to have the type `int expr`.
  + Need a way to inform the compiler that `Bool true` has type `bool value`.

## Generalized Algebraic Data Types

GADTs allow us to **refine** the return type of the data constructor. 

In [93]:
type 'a value =
  | Int : int -> int value
  | Bool : bool -> bool value
  
type 'a expr =
  | Val : 'a value -> 'a expr
  | Plus : int expr * int expr -> int expr
  | Mult : int expr * int expr -> int expr
  | Ite : bool expr * 'a expr * 'a expr -> 'a expr

type 'a value = Int : int -> int value | Bool : bool -> bool value


type 'a expr =
    Val : 'a value -> 'a expr
  | Plus : int expr * int expr -> int expr
  | Mult : int expr * int expr -> int expr
  | Ite : bool expr * 'a expr * 'a expr -> 'a expr


## Evaluator remains the same

In [95]:
let rec eval : type a. a expr -> a val_ = 
  fun e -> match e with
  | Val (Int i) -> Int i
  | Val (Bool i) -> Bool i
  | Plus (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 + i2)
  | Mult (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 * i2)
  | Ite (p,e1,e2) ->
    let Bool b = eval p in
    if b then eval e1 else eval e2

val eval : 'a expr -> 'a val_ = <fun>


In [97]:
eval @@ Plus (Val (Bool true), Val (Int 10))

error: compile_error

## GADTs are very powerful!

* Features
  + Refine return types.
  + Introduce existential types.

* Some uses
  + Length indexed vectors.
  + Encoding Red-Black property, 2-3 property.
  + Encode type safety in lambda calculus.

## Church Numerals

* Many of the shape properties of data structure involve some integers. 
* Let's encode natural numbers using GADTs.
  + This is the familiar Church encoding.

In [98]:
type z = Z
type 'n s = S : 'n -> 'n s

type z = Z


type 'n s = S : 'n -> 'n s


In [100]:
S (S (S Z))

- : z s s s = S (S (S Z))


In [ ]:
## Length-indexed 

Consider the untyped lambda calulus AST.

In [2]:
type expr =
  | Var of string
  | Lam of string * expr
  | App of expr * expr

type expr = Var of string | Lam of string * expr | App of expr * expr


For good measure, let us also enrich the calulus with integers, booleans and pairs and operations on those data types. 

In [3]:
type expr =
  | Var of string
  | Lam of string * expr
  | App of expr * expr
  (* Integers *)
  | Int of int
  | Plus of expr * expr
  | Mult of expr * expr
  (* Booleans *)
  | Bool of bool
  | Ite of expr * expr * expr
  (* Pairs *)
  | Pair of expr * expr
  | Fst of expr
  | Snd of expr

type expr =
    Var of string
  | Lam of string * expr
  | App of expr * expr
  | Int of int
  | Plus of expr * expr
  | Mult of expr * expr
  | Bool of bool
  | Ite of expr * expr * expr
  | Pair of expr * expr
  | Fst of expr
  | Snd of expr


At this point, you could easily imagine extending your assignment 2 to interpret the additional terms. However, this interpreter will have to handle lots of non sensical expressions such as:

In [4]:
let e1 = Plus (Bool true, Int 10)

val e1 : expr = Plus (Bool true, Int 10)


where the interpreter will have to raise an error. In $\lambda^{\rightarrow}$, we studied how to tag such expressions **ill-typed**.

OCaml language has support for more advanced algebraic data type (ADTs) definition called the **Generalised Algebraic Data Types** or **GADTs**. Similar to ADTs, GADTs allow you to define new sum types $'a ~t$. But unlike ADTs, the GADTs can specify a differnt $'a$ for each constructor. This idea is very powerful. 

We can encode the typing rules for well-typed expressions in the AST for our extended lambda calculus.

In [8]:
type (_,_) equal = Refl : ('a,'a) equal

type (_, _) equal = Refl : ('a, 'a) equal
